# Unión de archivos de series temporales de humedad del suelo -10cm desde repositorio Cassandra

> Elaborado por Paola Álvarez, profesional contratista IDEAM, contrato 196 de 2024. Comentarios o inquietudes, remitir a *palvarez@ideam.gov.co* 

In [1]:
import pandas as pd
import numpy as np
import datetime
import prestodb
import statistics
import os
import fnmatch

## Unir todos los datos de temperatura del suelo

In [3]:
### Se crea un diccionario con los nombres de archivos como "keys" y rutas
file_paths = {}
for root, dirs, files in os.walk('.'):
    # Se añade condición de inicio en '20' para que solo tome las carpetas de interés
    if os.path.basename(root).startswith(('20','19')):
        for f in files:
            if f.endswith('.csv'):
                if f not in file_paths:
                    file_paths[f] = []
                file_paths[f].append(root)

# para cada archivo en el diccionario, se busca en cada carpeta y se unen los archivos con el
# mismo nombre al inicio del directorio
for f, paths in file_paths.items():
    txt = []
    header_found = False  # Variable para indicar si ya se encontró la fila de encabezados

    for p in paths:
        with open(os.path.join(p, f), encoding='latin-1') as f2:
            lines = f2.readlines()
            if not header_found:
                # Agregar la primera línea (fila de encabezados) solo una vez
                txt.append(lines[0])
                header_found = True

            txt.extend(lines[1:])  # Agregar el resto de líneas

    try:
        with open(f, 'w') as f3:
            f3.write(''.join(txt))
    except UnicodeEncodeError as e:
        print("Error de codificación en el archivo:", f)
        print("Posición del caracter:", e.start)

## Corrección de errores

Si aplica, a continuación se corrigen los errores reportados en el anterior código en el bloque try-except.

In [31]:
# Se observan los directorios en os que se encuentran los archivos de la estación
for root, dirs, files in os.walk('.'):
    for f in files:
        if f.endswith('Estacion_0011115501.csv'):
            print(root)

.
.\2017-01_a_2017-12
.\2018-01_a_2018-12


In [33]:
# Al avidenciar que trata de un error de encoding, se indaga en el directorio desde el que puede iniciar el error (usualmente
# desde cierto periodo se generan errores de encoding que persisten
archivo_defectuoso = pd.read_csv('2018-01_a_2018-12/Estacion_0011115501.csv', encoding='latin-1')
archivo_defectuoso['Name'].unique()

array(['CAÃ\x91ASGORDAS'], dtype=object)

In [6]:
# Caso 1: solo hay error en unas cuantas carpetas. El reemplazado del error de encoding se hace en las carpetas específicas.
def findReplace(directory, find, replace, filePattern, targetFileName=None):
    for path, dirs, files in os.walk(os.path.abspath(directory)):
        for filename in fnmatch.filter(files, filePattern):
            # Verificar si el targetFileName es None o si el filename coincide con targetFileName
            if targetFileName is None or filename == targetFileName:
                filepath = os.path.join(path, filename)
                with open(filepath, encoding='latin-1') as f: #'utf-8' 'latin-1'
                    s = f.read()
                s = s.replace(find, replace)
                try:
                    # Se utiliza un bloque try para manejar errores durante la escritura
                    with open(filepath, "w", encoding='latin-1') as f:
                        f.write(s)
                except Exception as e:
                    # Puedes manejar aquí errores específicos si lo necesitas
                    print(f"Error escribiendo en el archivo {filename}: {e}")

In [7]:
findReplace("2024-06_a_2024-07", "UNIVERSIDAD SANTO TOMÃ\x81S", "UNIVERSIDAD SANTO TOMÁS", "*.csv", targetFileName="Estacion_0021205523.csv")

In [35]:
# Caso 2: Error de encoding en todas las carpetas; el reemplazado se hace en todas las carpetas
def findReplace_alldicts(directory, find, replace, filePattern, targetFileName=None):
    for path, dirs, files in os.walk(os.path.abspath(directory)):
        for filename in files:
            # Verificar si el filename coincide con targetFileName
            if filename == targetFileName:
                filepath = os.path.join(path, filename)
                with open(filepath, encoding='latin-1') as f:
                    s = f.read()
                s = s.replace(find, replace)
                try:
                    # Intenta escribir los cambios al mismo archivo
                    with open(filepath, "w", encoding='latin-1') as f:
                        f.write(s)
                except Exception as e:
                    # Maneja posibles errores de escritura
                    print(f"Error escribiendo en el archivo {filename}: {e}")

In [37]:
findReplace_alldicts(r'C:\Users\palvarez\Documents\clone\OE_CTO196-2024\OE_3_QC_Variables\4_HumedadSuelo\HRS10',
                     "CAÃ\x91ASGORDAS", "CAÑASGORDAS", "*.csv", targetFileName="Estacion_0011115501.csv")

In [27]:
# Función para unir los archivos de las estaciones con los errores detectados
def JoinFixedStations(fileName):
    
    file_paths = {}
    for root, dirs, files in os.walk('.'):
        # Se añade condición de inicio en '20' para que solo tome las carpetas de interés
        if os.path.basename(root).startswith('20'):
            for f in files:
                if f.endswith(fileName):
                    if f not in file_paths:
                        file_paths[f] = []
                    file_paths[f].append(root)
    
    for f, paths in file_paths.items():
        txt = []
        header_found = False # Variable para observar si hay encabezado
        
        for p in paths:
            with open(os.path.join(p, f), encoding='latin-1') as f2: #'latin-1','utf-8'
                lines = f2.readlines()
                if not header_found:
                # Agregar la primera línea (fila de encabezados) solo una vez
                    txt.append(lines[0])
                    header_found = True
                
                txt.extend(lines[1:])
                
        try:
            with open(f, 'w') as f3:
                f3.write(''.join(txt))
        except UnicodeEncodeError as e:
            print("Error en el archivo:", f)
            print('Directorio', paths)
            print("Posición del caracter:", e.start)
        except UnicodeEncodeError as e:
            print("Error de codificación en el archivo:", f)
            print("Posición del caracter:", e.start)

In [39]:
JoinFixedStations('Estacion_0011115501.csv')